In [1]:
import cv2 
import os
from PIL import Image
import numpy as np

In [2]:
image_directory='dataset/'
no_cancer_images=os.listdir(image_directory+ 'no/')
yes_cancer_images=os.listdir(image_directory+ 'cancer/')
dataset=[]
label=[]
INPUT_SIZE=64

In [3]:
# checking image is jpg or not and making dataset and label
for i,image_name in enumerate(no_cancer_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'no/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE)) 
        dataset.append(np.array(image))
        label.append(0)
for i , image_name in enumerate(yes_cancer_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'cancer/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE)) 
        dataset.append(np.array(image))
        label.append(1)

In [4]:
dataset=np.array(dataset)
label=np.array(label)
label.shape

(912,)

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(dataset,label,test_size=0.2,random_state=0)

In [6]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize

In [7]:
# normalizing data for binary classification
x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)

In [8]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation,Dropout,Flatten,Dense

In [9]:
# model building

#sequential model
model=Sequential()
#firstlayer: putting shape of the input
#Conv2D(filter,kersnal_size)using 32 no. of convolution and
# using (3,3) kernal size and input size of 64x64
model.add(Conv2D(32,(3,3), input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
#adding activation fuction
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#initializing kernel
model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#flattening
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#output layer using 1 coz using binary classification prob 
#if we used categorical classification we would have used 2
model.add(Dense(1))
#using sigmoid function as binary cross entropy 
#if categorical cross entropy we use softmax
model.add(Activation('sigmoid'))

#compiling the model using binary_cross_entrpy
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer='adam',metrics=['accuracy'])  

#fitting the model
model.fit(x_train,y_train,batch_size=32, verbose=1,epochs=20, validation_data=(x_test,y_test),shuffle=False)
#saving model
model.save('cancer10Epochs.h5')

Epoch 1/20


c:\anaconda\lib\site-packages\keras\backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


23/23 [==============================] - 3s 109ms/step - loss: 0.4869 - accuracy: 0.8025 - val_loss: 0.3925 - val_accuracy: 0.9016
Epoch 2/20
23/23 [==============================] - 2s 93ms/step - loss: 0.4545 - accuracy: 0.8395 - val_loss: 0.3790 - val_accuracy: 0.9016
Epoch 3/20
23/23 [==============================] - 2s 95ms/step - loss: 0.4280 - accuracy: 0.8409 - val_loss: 0.3679 - val_accuracy: 0.9016
Epoch 4/20
23/23 [==============================] - 2s 93ms/step - loss: 0.4050 - accuracy: 0.8395 - val_loss: 0.3786 - val_accuracy: 0.9016
Epoch 5/20
23/23 [==============================] - 2s 83ms/step - loss: 0.3812 - accuracy: 0.8368 - val_loss: 0.3712 - val_accuracy: 0.9016
Epoch 6/20
23/23 [==============================] - 2s 87ms/step - loss: 0.3660 - accuracy: 0.8422 - val_loss: 0.3688 - val_accuracy: 0.9071
Epoch 7/20
23/23 [==============================] - 2s 88ms/step - loss: 0.3305 - accuracy: 0.8409 - val_loss: 0.2602 - val_accuracy: 0.9071
Epoch 8/20
23/23 [=====

In [10]:
from keras.models import load_model

model1=load_model('cancer10Epochs.h5')
image=cv2.imread('C:\\Users\\Ujjwal Shandilya\\Desktop\\cancer\\cancer_datasest\\dataset\\cancer\\benign (277)-sharpened.jpg')
img=Image.fromarray(image)
img=img.resize((64,64))
img=np.array(img)
img=np.expand_dims(img,axis=0)
result=model.predict(img)
print(result[0][0])

1/1 [==============================] - 0s 73ms/step
1.0
